Notebook dealing with the final preprocessing steps for the neural networks.


In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.utils.data
import pandas as pd
from torch.utils import data
from numpy import array
from numpy import argmax
import argparse
from torch.autograd import Variable
from torch import optim
import numpy as np
import os
from sklearn.model_selection import train_test_split
import logging
import pickle as pkl
import warnings
pd.options.mode.chained_assignment = None
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import glob
import random
random.seed(1991)
#torch.set_default_tensor_type('torch.cuda.DoubleTensor')

In [2]:
torch.manual_seed(1991)
if torch.cuda.is_available():
    
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda")

In [3]:
heads=pd.read_pickle("/data/dharp/compounding/datasets/heads_CompoundCentric_DecadeAgnostic_300.pkl")
heads.info()
heads.head()

<class 'pandas.core.frame.DataFrame'>
Index: 53466 entries, a_n to zygote_n
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 122.8+ MB


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
head,,,,,,,,,,,,,,,,,,,,,
a_n,0.440430,0.005506,0.002429,-0.002683,-0.000983,-0.025453,0.194777,-0.030807,-0.009806,-0.103889,...,-0.028331,0.027137,-0.043847,0.006858,0.008193,0.015432,0.018965,-0.011933,-0.015698,0.012663
aa_n,0.011832,0.002295,-0.000676,0.000938,0.000343,-0.002899,0.050492,-0.017074,0.000098,0.051071,...,-0.038320,-0.032570,-0.009443,-0.000768,0.014299,0.025628,0.049950,-0.026634,0.003430,0.018899
aaa_n,0.313449,-0.016028,0.011828,0.002696,-0.000780,-0.016718,0.154211,-0.031180,0.108424,-0.081133,...,-0.046221,-0.058067,-0.012101,0.011162,0.036212,0.029292,0.029706,-0.039024,0.025404,0.016696
aaaaa_n,0.000827,0.000443,0.000095,0.000450,-0.000050,-0.000347,0.004473,-0.000884,0.001146,-0.005078,...,-0.044826,-0.022707,-0.011827,0.016305,0.012899,0.027791,0.047539,0.007618,0.024552,0.012872
aaai_n,0.000850,0.000880,0.000401,0.002374,0.004960,0.001507,0.003561,-0.000046,0.034051,-0.004198,...,0.023478,0.040822,-0.095591,0.044083,-0.010137,-0.067752,0.141350,0.147728,-0.015423,0.071274


In [4]:
modifiers=pd.read_pickle("/data/dharp/compounding/datasets/modifiers_CompoundCentric_DecadeAgnostic_300.pkl")
modifiers.info()
modifiers.head()

<class 'pandas.core.frame.DataFrame'>
Index: 54400 entries, a_n to zylobalsamum_n
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 124.9+ MB


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
modifier,,,,,,,,,,,,,,,,,,,,,
a_n,0.339999,0.032878,-0.001667,0.121973,-0.022126,-0.016050,0.114271,-0.025870,0.052949,-0.093165,...,-0.025719,-0.043307,0.013846,0.022528,-0.038733,0.035699,0.037341,0.012729,0.017356,-0.005785
aa_n,0.192280,0.011892,0.002212,-0.001564,0.036605,-0.006458,0.050666,-0.014895,0.001873,-0.046352,...,-0.056977,-0.029964,-0.027422,0.010024,0.043846,0.018267,0.062345,0.030111,0.025396,-0.001170
aaa_n,0.065267,0.047112,0.000272,0.091206,-0.015523,-0.003515,0.003856,-0.001949,-0.005680,-0.005168,...,-0.039798,-0.023954,-0.025154,0.012599,0.021654,0.034694,0.046989,0.000299,0.021891,-0.000029
aaaa_n,0.000827,0.000443,0.000095,0.000450,-0.000050,-0.000347,0.004473,-0.000884,0.001146,-0.005078,...,-0.044826,-0.022707,-0.011827,0.016305,0.012899,0.027791,0.047539,0.007618,0.024552,0.012872
aaaaa_n,0.000827,0.000443,0.000095,0.000450,-0.000050,-0.000347,0.004473,-0.000884,0.001146,-0.005078,...,-0.044826,-0.022707,-0.011827,0.016305,0.012899,0.027791,0.047539,0.007618,0.024552,0.012872


In [5]:
novel_compounds_list = pkl.load( open( "/data/dharp/compounding/datasets/novel_compounds_list.pkl", "rb" ) )
m, h = zip(*novel_compounds_list)
heads_list=list(set(h))
modifiers_list=list(set(m))

In [6]:
novel_compounds=pd.DataFrame(novel_compounds_list)
novel_compounds.columns=['modifier','head']

In [7]:
positive_heads=pd.merge(novel_compounds.drop('modifier',axis=1),heads.reset_index(),on=["head"])

positive_modifiers=pd.merge(novel_compounds.drop('head',axis=1),modifiers.reset_index(),on=["modifier"])
#positive_df['Plausibility']=True

In [8]:
positive_heads.head()

,head,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,peaceful_n,0.094746,0.394519,0.049438,-0.055479,0.067019,0.017066,-0.067276,-0.005653,0.039678,...,0.008639,-0.016868,-0.025363,0.001678,0.022061,0.004521,-0.008992,0.005364,-0.005843,0.003950
1,peaceful_n,0.094746,0.394519,0.049438,-0.055479,0.067019,0.017066,-0.067276,-0.005653,0.039678,...,0.008639,-0.016868,-0.025363,0.001678,0.022061,0.004521,-0.008992,0.005364,-0.005843,0.003950
2,robinson_n,0.516893,-0.027680,-0.009162,0.021842,-0.002822,-0.023920,0.113440,-0.009606,-0.013875,...,0.087203,-0.020085,-0.069696,0.059515,-0.203745,-0.053035,0.066117,-0.016516,0.295535,0.011347
3,robinson_n,0.516893,-0.027680,-0.009162,0.021842,-0.002822,-0.023920,0.113440,-0.009606,-0.013875,...,0.087203,-0.020085,-0.069696,0.059515,-0.203745,-0.053035,0.066117,-0.016516,0.295535,0.011347
4,supply_n,0.849265,-0.049544,-0.023528,-0.010386,0.010196,-0.052200,0.357877,-0.056204,-0.047523,...,0.025304,-0.000088,-0.022895,0.004955,0.006311,-0.000669,-0.005519,-0.011278,0.000614,-0.017954


In [9]:
positive_heads_tensor = torch.tensor(positive_heads.drop('head',axis=1).values)
positive_heads_tensor.shape

torch.Size([28448, 300])

In [10]:
positive_modifiers_tensor = torch.tensor(positive_modifiers.drop('modifier',axis=1).values)
positive_modifiers_tensor.shape

torch.Size([28448, 300])

In [11]:
positive_Y=torch.ones(positive_modifiers_tensor.shape[0])
positive_Y.shape

torch.Size([28448])

In [12]:
positive_class=torch.cat((positive_modifiers_tensor, positive_heads_tensor), 1)
positive_class.shape

torch.Size([28448, 600])

In [13]:
def neg_df_creator(file):
    pkl_file=pkl.load( open(file,'rb'))
    df=pd.DataFrame(pkl_file)
    
    df.columns=['modifier','head']
    negative_heads=pd.merge(df.drop('modifier',axis=1),heads.reset_index(),on=["head"])
    negative_modifiers=pd.merge(df.drop('head',axis=1),modifiers.reset_index(),on=["modifier"])
    negative_heads_tensor = torch.tensor(negative_heads.drop('head',axis=1).values)
    negative_modifiers_tensor = torch.tensor(negative_modifiers.drop('modifier',axis=1).values)

    negative_Y=torch.zeros(negative_modifiers_tensor.shape[0])
    negative_class=torch.cat((negative_modifiers_tensor, negative_heads_tensor), 1)

    return negative_class,negative_Y

In [14]:
def tensor_joiner(files):
    tensor_list=[]
    for file in files:
        negative_class,negative_Y=neg_df_creator(file)
        X=torch.cat((positive_class, negative_class), 0)
        Y=torch.cat((positive_Y,negative_Y),0)
        tensor_list.append([X,Y])
    
    return tensor_list

In [15]:
corrupt_modifier_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_modifier*"):
    corrupt_modifier_files.append(file)
corrupt_modifiers=tensor_joiner(corrupt_modifier_files)

In [16]:
corrupt_head_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_head*"):
    corrupt_head_files.append(file)
corrupt_heads=tensor_joiner(corrupt_head_files)

In [17]:
input_size = 600
hidden_size = 300
num_classes = 2
num_epochs = 50
batch_size = 100
learning_rate = 0.001

In [18]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes,bias=False)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [19]:
def looper(datasets):
    total_accuracy=[]
    
    for dataset in datasets:
        X=dataset[0]
        Y=dataset[1]
        n = len(X)  # how many total elements you have
        n_test = int( n * .2 )  # number of test/val elements
        n_train = n - n_test
        idx = list(range(n))  # indices to all elements
        random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
        train_idx = idx[:n_train]
        test_idx = idx[n_train:]
        trX=X[train_idx].float().to(device)
        teX=X[test_idx].float().to(device)

        trY=Y[train_idx].long().to(device)
        teY=Y[test_idx].long().to(device)
        model = NeuralNet(input_size, hidden_size, num_classes).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        n_examples=trX.shape[0]
        for i in range(num_epochs):

            cost = 0.

            num_batches = n_examples // batch_size
            for k in range(num_batches):
                start, end = k * batch_size, (k + 1) * batch_size
                outputs = model(trX[start:end])
                loss = criterion(outputs, trY[start:end])

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            #if (k+1) % 100 == 0:
            #print ('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, num_epochs, loss.item()))

        with torch.no_grad():
            correct=0
            total=0
            outputs = model(teX)
            _, predicted = torch.max(outputs.data, 1)
            total += teY.size(0)
            correct += (predicted == teY).sum().item()
        curr_acc=100 * correct / total
        print(curr_acc)
        total_accuracy.append(curr_acc)
    return total_accuracy
        

In [20]:
cor_head_acc=looper(corrupt_heads)

82.91589770630108
83.12681254943317
82.59952544160295
82.76649969241585
83.14438878636084
83.05650760172247
82.90710958783724
83.22348185253537
82.67861850777749
82.74892345548818


In [21]:
round(np.mean(cor_head_acc),2)

82.92

In [22]:
round(np.std(cor_head_acc),2)

0.2

In [23]:
cor_mod_acc=looper(corrupt_modifiers)

82.88074523244573
83.40803234027595
82.29194129536866
81.87889972756832
82.72255910009667
82.21284822919412
82.45891554618156
82.5643729677476
82.6610422708498
82.3007294138325


In [24]:
round(np.mean(cor_mod_acc),2)

82.54

In [25]:
round(np.std(cor_mod_acc),2)

0.4